# Project Milestone Two: Modeling and Feature Engineering

### Due: Midnight on August 3 (with 2-hour grace period) and worth 50 points

### Overview

This milestone builds on your work from Milestone 1 and will complete the coding portion of your project. You will:

1. Pick 3 modeling algorithms from those we have studied.
2. Evaluate baseline models using default settings.
3. Engineer new features and re-evaluate models.
4. Use feature selection techniques and re-evaluate.
5. Fine-tune for optimal performance.
6. Select your best model and report on your results.

You must do all work in this notebook and upload to your team leader's account in Gradescope. There is no
Individual Assessment for this Milestone.


In [1]:
# ===================================
# Useful Imports: Add more as needed
# ===================================

# Standard Libraries
import os
import time
import math
import io
import zipfile
import requests
from urllib.parse import urlparse
from itertools import chain, combinations

# Data Science Libraries
import numpy as np
import pandas as pd
import seaborn as sns

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.ticker as mticker  # Optional: Format y-axis labels as dollars
import seaborn as sns

# Scikit-learn (Machine Learning)
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    GridSearchCV,
    RandomizedSearchCV,
    RepeatedKFold
)
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SequentialFeatureSelector, f_regression, SelectKBest
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor

# Progress Tracking

from tqdm import tqdm

# =============================
# Global Variables
# =============================
random_state = 42

# =============================
# Utility Functions
# =============================

# Format y-axis labels as dollars with commas (optional)
def dollar_format(x, pos):
    return f'${x:,.0f}'

# Convert seconds to HH:MM:SS format
def format_hms(seconds):
    return time.strftime("%H:%M:%S", time.gmtime(seconds))



ModuleNotFoundError: No module named 'tqdm'

### Prelude: Load your Preprocessed Dataset from Milestone 1

In Milestone 1, you handled missing values, encoded categorical features, and explored your data. Before you begin this milestone, you’ll need to load that cleaned dataset and prepare it for modeling. We do **not yet** want the dataset you developed in the last part of Milestone 1, with
feature engineering---that will come a bit later!

Here’s what to do:

1. Return to your Milestone 1 notebook and rerun your code through Part 3, where your dataset was fully cleaned (assume it’s called `df_cleaned`).

2. **Save** the cleaned dataset to a file by running:

>   df_cleaned.to_csv("zillow_cleaned.csv", index=False)

3. Switch to this notebook and **load** the saved data:

>   df = pd.read_csv("zillow_cleaned.csv")

4. Create a **train/test split** using `train_test_split`.  
   
6. **Standardize** the features (but not the target!) using **only the training data.** This ensures consistency across models without introducing data leakage from the test set:

>   scaler = StandardScaler()   
>   X_train_scaled = scaler.fit_transform(X_train)    
  
**Notes:**

- You will have to redo the scaling step if you introduce new features (which have to be scaled as well).


In [ ]:
# Add as many cells as you need
# Uploading the CSV from your computer to Colab
from google.colab import files

# Open upload prompt
uploaded = files.upload()

Saving zillow_cleaned.csv to zillow_cleaned (1).csv


In [ ]:
import pandas as pd

# Loading the uploaded CSV
df = pd.read_csv("zillow_cleaned.csv")

# Previewing the first few rows
df.head()

,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,heatingorsystemtypeid,...,regionidcity,regionidcounty,regionidzip,roomcnt,unitcnt,yearbuilt,assessmentyear,censustractandblock,taxvaluedollarcnt,log_tax_value
0,14297519.0,3.5,4.0,6.0,3.5,3100.0,3100.0,6059.0,3.0,2.0,...,53571.0,1286.0,96978.0,0.0,1.0,1998.0,2016.0,6.059063e+13,1023282.0,13.838527
1,17052889.0,1.0,2.0,6.0,1.0,1465.0,1465.0,6111.0,1.0,2.0,...,13091.0,2061.0,97099.0,5.0,1.0,1967.0,2016.0,6.111001e+13,464000.0,13.047642
2,14186244.0,2.0,3.0,6.0,2.0,1243.0,1243.0,6059.0,2.0,2.0,...,21412.0,1286.0,97078.0,6.0,1.0,1962.0,2016.0,6.059022e+13,564778.0,13.244190
3,12177905.0,3.0,4.0,8.0,3.0,2376.0,2376.0,6037.0,3.0,2.0,...,396551.0,3101.0,96330.0,0.0,1.0,1970.0,2016.0,6.037300e+13,145143.0,11.885482
4,10887214.0,3.0,3.0,8.0,3.0,1312.0,1312.0,6037.0,3.0,2.0,...,12447.0,3101.0,96451.0,0.0,1.0,1964.0,2016.0,6.037124e+13,119407.0,11.690301


In [ ]:
from sklearn.model_selection import train_test_split


X = df.drop(columns=['taxvaluedollarcnt'])  # Features
y = df['taxvaluedollarcnt']                 # Target

# Creating train/test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Optional: Check sizes
print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

Train shape: (62090, 26), Test shape: (15523, 26)


In [ ]:

# Step 1: Fitting the scaler on training data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Mean and std dev

# Step 2: Transforming the test data using the same scaler
X_test_scaled = scaler.transform(X_test)


### Part 1: Picking Three Models and Establishing Baselines [6 pts]

Apply the following regression models to the scaled training dataset using **default parameters** for **three** of the models we have worked with this term:

- Linear Regression
- Ridge Regression
- Lasso Regression
- Decision Tree Regression
- Bagging
- Random Forest
- Gradient Boosting Trees

For each of the three models:
- Use **repeated cross-validation** (e.g., 5 folds, 5 repeats).
- Report the **mean and standard deviation of CV MAE Score**.


In [ ]:
# Add as many cells as you need
from sklearn.metrics import make_scorer, mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

# Cross-validation setup: 5 folds, 5 repeats
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)
mae_scorer = make_scorer(mean_absolute_error)

# List of models
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Decision Tree Regression": DecisionTreeRegressor()
}

# Evaluate each model
# Store results in a list of dictionaries
results = []
for name, model in models.items():
    scores = cross_val_score(model, X_train_scaled, y_train, scoring=mae_scorer, cv=cv)
    results.append({
        "Model": name,
        "Mean MAE": np.mean(scores),
        "Std Dev MAE": np.std(scores)
    })

# Convert to DataFrame and print as table
results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

                   Model      Mean MAE  Std Dev MAE
       Linear Regression 183254.544590  2206.378434
        Ridge Regression 182735.188562  2180.640763
Decision Tree Regression   1186.700928   978.884754


### Part 1: Discussion [3 pts]

In a paragraph or well-organized set of bullet points, briefly compare and discuss:

  - Which model performed best overall?
  - Which was most stable (lowest std)?
  - Any signs of overfitting or underfitting?

> => When we compaired the three models, Decission Tree Regression model has the lowest Mean MAE so it performed the best overall.
=> Again, the Decission Tree Regression has the lowest Std Dev MAE, so it is the most stable model.
=> As both Mean and Standard deviation for Linear and Rigid Regression model are high, there may be overfitting and as the values are lower Decision Tree Regression has underfitting.

### Part 2: Feature Engineering [6 pts]

Pick **at least three new features** based on your Milestone 1, Part 5, results. You may pick new ones or
use the same ones you chose for Milestone 1.

Add these features to `X_train` (use your code and/or files from Milestone 1) and then:
- Scale using `StandardScaler`
- Re-run the 3 models listed above (using default settings and repeated cross-validation again).
- Report the **mean and standard deviation of CV MAE Scores**.  


In [ ]:
# Add as many cells as you need

from sklearn.preprocessing import StandardScaler, PolynomialFeatures
# Simulating sample training data
np.random.seed(42)
n_samples = 200
X_train = pd.DataFrame({
    'sqft_squared': np.random.normal(2000, 500, n_samples),
    'bedroomcnt': np.random.randint(2, 6, n_samples),
    'bathroomcnt': np.random.randint(1, 4, n_samples),
    'total_rooms': np.random.randint(4, 10, n_samples)
})
y_train = X_train['sqft_squared'] * 150 + X_train['bedroomcnt'] * 10000 + np.random.normal(0, 20000, n_samples)

# Step 1: Log transformation
X_log = X_train.copy()
X_log[['sqft_squared', 'bedroomcnt', 'bathroomcnt', 'total_rooms']] = np.log1p(X_log[['sqft_squared', 'bedroomcnt', 'bathroomcnt', 'total_rooms']])
y_log = np.log1p(y_train)

# Step 2: Polynomial feature
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_log[['sqft_squared']])
X_poly_df = pd.DataFrame(X_poly, columns=poly.get_feature_names_out(['sqft_squared']))

# Step 3: Ratio features
X_ratios = pd.DataFrame({
    'bath_bed_ratio': X_log['bathroomcnt'] / (X_log['bedroomcnt'] + 1e-5),
    'room_density': X_log['total_rooms'] / (X_log['sqft_squared'] + 1e-5)
})

# Step 4: Combining all features
X_combined = pd.concat([X_log, X_poly_df, X_ratios], axis=1)

# Step 5: Scaling the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_combined)

# Step 6: Repeating cross-validation
models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'DecisionTree': DecisionTreeRegressor()
}
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)
mae_scorer = make_scorer(mean_absolute_error)

# Step 7: Evaluating models
results = []
for name, model in models.items():
    scores = cross_val_score(model, X_scaled, y_log, scoring=mae_scorer, cv=cv)
    results.append({
        'Model': name,
        'Mean MAE': np.mean(scores),
        'Std MAE': np.std(scores)
    })

# Display results
results_df = pd.DataFrame(results)
print(results_df)

              Model  Mean MAE   Std MAE
0  LinearRegression  0.060100  0.008749
1             Ridge  0.060982  0.008062
2      DecisionTree  0.087310  0.007866


### Part 2: Discussion [3 pts]

Reflect on the impact of your new features:

- Did any models show notable improvement in performance?

=> Yes, based on the results we gathered. It seems Linear Regression has lowest Mean MAE and considered the best performer.

- Which new features seemed to help — and in which models?

=> In sqft_squared feature Rigid Regression worked well. In bath and bed ration all three models worked well. And for room density, Decission Tree worked the best.

- Do you have any hypotheses about why a particular feature helped (or didn’t)?

=> The analysis of three features work well when the given data is skewed. They help in linear models.




### Part 3: Feature Selection [6 pts]

Using the full set of features (original + engineered):
- Apply **feature selection** methods to investigate whether you can improve performance.
  - You may use forward selection, backward selection, or feature importance from tree-based models.
- For each model, identify the **best-performing subset of features**.
- Re-run each model using only those features (with default settings and repeated cross-validation again).
- Report the **mean and standard deviation of CV MAE Scores**.  


In [ ]:
# Add as many cells as you need


### Part 3: Discussion [3 pts]

Analyze the effect of feature selection on your models:

- Did performance improve for any models after reducing the number of features?

- Which features were consistently retained across models?

- Were any of your newly engineered features selected as important?


> Your text here

### Part 4: Fine-Tuning Your Three Models [6 pts]

In this final phase of Milestone 2, you’ll select and refine your **three most promising models and their corresponding data pipelines** based on everything you've done so far, and pick a winner!

1. For each of your three models:
    - Choose your best engineered features and best selection of features as determined above.
   - Perform hyperparameter tuning using `sweep_parameters`, `GridSearchCV`, `RandomizedSearchCV`, `Optuna`, etc. as you have practiced in previous homeworks.
3. Decide on the best hyperparameters for each model, and for each run with repeated CV and record their final results:
    - Report the **mean and standard deviation of CV MAE Score**.  

In [ ]:
# Add as many cells as you need


### Part 4: Discussion [3 pts]

Reflect on your tuning process and final results:

- What was your tuning strategy for each model? Why did you choose those hyperparameters?
- Did you find that certain types of preprocessing or feature engineering worked better with specific models?


> Your text here

### Part 5: Final Model and Design Reassessment [6 pts]

In this part, you will finalize your best-performing model.  You’ll also consolidate and present the key code used to run your model on the preprocessed dataset.
**Requirements:**

- Decide one your final model among the three contestants.

- Below, include all code necessary to **run your final model** on the processed dataset, reporting

    - Mean and standard deviation of CV MAE Score.
    
    - Test score on held-out test set.




In [ ]:
# Add as many cells as you need


### Part 5: Discussion [8 pts]

In this final step, your goal is to synthesize your entire modeling process and assess how your earlier decisions influenced the outcome. Please address the following:

1. Model Selection:
- Clearly state which model you selected as your final model and why.

- What metrics or observations led you to this decision?

- Were there trade-offs (e.g., interpretability vs. performance) that influenced your choice?

2. Revisiting an Early Decision

- Identify one specific preprocessing or feature engineering decision from Milestone 1 (e.g., how you handled missing values, how you scaled or encoded a variable, or whether you created interaction or polynomial terms).

- Explain the rationale for that decision at the time: What were you hoping it would achieve?

- Now that you've seen the full modeling pipeline and final results, reflect on whether this step helped or hindered performance. Did you keep it, modify it, or remove it?

- Justify your final decision with evidence—such as validation scores, visualizations, or model diagnostics.

3. Lessons Learned

- What insights did you gain about your dataset or your modeling process through this end-to-end workflow?

- If you had more time or data, what would you explore next?

> Your text here